In [10]:
import pandas as pd
import json
import re
import yaml
import logging

In [11]:
# Set up logging
logging.basicConfig(level=logging.INFO)

## Primera evaluación

### Cargamos todos los datos de las preguntas Reales

In [36]:
# Union de los DataFrames de resultados

# Función para cargar datos desde un archivo JSON a un DataFrame
def load_json_to_dataframe(filename):
    with open(filename, 'r', encoding='utf-8') as file:
        data = json.load(file)
    df = pd.DataFrame(data)
    return df

# Lista de archivos JSON
json_files = ['resultados.json', 'resultados2.json', 'resultados3.json']

# Cargar los DataFrames de cada archivo JSON y concatenarlos
dataframes = [load_json_to_dataframe(file) for file in json_files]
df_results = pd.concat(dataframes, ignore_index=True)

In [59]:
# Cargamos las respuestas de expertos
with open('ground_truth.json', 'r') as file:
    ground_truth_dict = json.load(file)
ground_truth_list = [entry['ground_truth'] for entry in ground_truth_dict]
df_results['ground_truth'] = ground_truth_list

In [69]:
df_results.head()

,question,result,contexts,duration,ground_truth
0,¿Usted realmente están involucrados en este pr...,"Rewrite\n\nAs an expert Q&A system, I must say...",Los investigadores del proyecto Magos nombraro...,18.563879,Muchas gracias por su pregunta. La Asamblea Ci...
1,Cuánto afecta a la temperatura ya la salud de ...,"Por favor, realice una pregunta adecuada.> \n>...",Multiplicar la producción de leguminosas\n\nLa...,5.072124,"En términos generales, las estrellas de conden..."
2,Porque hay empresas que para ganarse más diner...,"Por favor, realice una pregunta adecuada.> \n>...",PROPUESTAS SOBRE EL TERRITORIO | Propuesta 2.1...,3.025464,Esta temática no forma parte de los ejes de tr...
3,La desembocadura de residuos tóxicos en los rí...,La transición energética en Cataluña hasta el ...,"LA LEY DE CAMBIO CLIMÁTICO\n\r\nEn Cataluña, l...",9.912813,La Agencia Catalana del Agua ejerce las funcio...
4,¿La ley que hizo por el cambio climático no fu...,No hay evidencia de que la lucha contra el cam...,"En su decisión, la Corte Suprema señaló que el...",17.057546,"La Ley 16/2017, de 1 de agosto, del cambio cli..."


In [103]:
# Descartamos las preguntas que han utilizado la búsqueda web, ya que implica una lógica más allá del query_engine de llamaindex
# y requeriría un desarrollo adicional

# Get sites where the web search engine was used
def load_custom(code):
    file_path = f'custom_{code}.yml'
    with open(file_path, 'r', encoding='utf-8') as file:
        templates = yaml.safe_load(file)
    return templates
custom_templates = load_custom('ES_01')
custom_templates
sites = custom_templates['SITES']

# Create a regular expression from the list of sites
pattern = '|'.join(sites)

# Filter out the results that contain the sites
filtered_df = df_results[~df_results['result'].str.contains(pattern, regex=True)].copy()

In [104]:
# Comprobamos que las preguntas que quedan son las basadas en los documentos de Weaviate o que se han
# respondido reusando contestar
filtered_df["result"].to_list()

['Por favor, realice una pregunta adecuada.> \n> Puede consultar información del proceso y las temáticas en la página web de la organización.    [[ES]](https://participa.gencat.cat/processes/assembleaclima?locale=es) |     [[CAT]](https://participa.gencat.cat/processes/assembleaclima)',
 'Por favor, realice una pregunta adecuada.> \n> Puede consultar información del proceso y las temáticas en la página web de la organización.    [[ES]](https://participa.gencat.cat/processes/assembleaclima?locale=es) |     [[CAT]](https://participa.gencat.cat/processes/assembleaclima)',
 'La transición energética en Cataluña hasta el año 2050 requiere un enfoque integral que abarque desde la descentralización y las energías renovables hasta la eficiencia energética y el autoconsumo. Sin embargo, la desembocadura de residuos tóxicos en los ríos es un tema crucial que nos lleva a reflexionar sobre la responsabilidad ambiental y social.\n\nEn este sentido, para enfrentar esta desafío, es fundamental implem

In [105]:
len(filtered_df)

38

In [106]:
filtered_df.rename(columns={'context': 'contexts'}, inplace=True)

### QueryEngine

In [12]:
from llama_index.llms.ollama import Ollama
from llama_index.vector_stores.weaviate import WeaviateVectorStore
from llama_index.core import VectorStoreIndex, Settings, ChatPromptTemplate
from llama_index.core.llms import ChatMessage, MessageRole
from llama_index.embeddings.text_embeddings_inference import TextEmbeddingsInference
import weaviate

import os

/home/brianda/projects/chatbots/dev_chatbot_assistant_for_citizen_assemblies/orchestator-venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [13]:
from dotenv import load_dotenv
dotenv_path = '../config.env'
load_dotenv(dotenv_path=dotenv_path)

True

In [14]:
# Load environment variables from config.env file, from the docker compose configuration
EMBED_MODEL_NAME = os.getenv('EMBED_MODEL_NAME')
EMBED_BASE_URL = os.getenv('EMBED_BASE_URL')
EMBED_TIMEOUT = str(os.getenv('EMBED_TIMEOUT'))
EMBED_BATCH_SIZE = str(os.getenv('EMBED_BATCH_SIZE'))

WEAVIATE_HOST = os.getenv('WEAVIATE_HOST')
WEAVIATE_PORT = '8090' #os.getenv('WEAVIATE_PORT')
WEAVIATE_GRPC_PORT = '50061' #os.getenv('WEAVIATE_GRPC_PORT')
WEAVIATE_URL = os.getenv('WEAVIATE_URL')

INDEX_NAME = os.getenv('INDEX_NAME')


In [15]:
# Function to set up the embeddings model
def setup_embedding_model():
    return TextEmbeddingsInference(
        model_name=EMBED_MODEL_NAME,
        base_url=EMBED_BASE_URL,
        timeout=EMBED_TIMEOUT, 
        embed_batch_size=EMBED_BATCH_SIZE
    )

# Function to configure settings for LLM and embedding model
def configure_settings(llm_model, embed_model):
    Settings.llm = llm_model
    Settings.embed_model = embed_model

# Function to connect to Weaviate
def connect_to_weaviate():
    logging.info(f"Iniciando la conexión a weaviate: {WEAVIATE_URL}")
    # It's necessary to indicate the grpc data especially if in docker compose it's redirected from a different port
    weaviate_client = weaviate.connect_to_custom(
                http_host=WEAVIATE_HOST,
                http_port=WEAVIATE_PORT,
                http_secure=False,
                grpc_host= 'localhost',
                grpc_port=WEAVIATE_GRPC_PORT,
                grpc_secure=False,
            )
    logging.info(f"Conectado a weaviate")
    return weaviate_client

# Function to initialize the query engine for Chat+RAG from Weaviate documents
def initialize_query_engine(weaviate_client, index_name, text_key="text"):
    vector_store = WeaviateVectorStore(weaviate_client=weaviate_client, 
                                       index_name=index_name,
                                       text_key=text_key)
    index = VectorStoreIndex.from_vector_store(vector_store)

    # Custom Text QA Prompt
    if index_name == INDEX_NAME:
        basic_content = custom_templates['BASIC_CONTENT_DOCS']
    else:
        basic_content = custom_templates['BASIC_CONTENT_WEB']   
    #basic_content = custom_templates['BASIC_CONTENT_DOCS'] if index_name == INDEX_NAME else custom_templates['BASIC_CONTENT_WEB']
    
    qa_prompt_str = custom_templates['QA_PROMPT_STR']

    chat_text_qa_msgs = [
        ChatMessage(
            role=MessageRole.SYSTEM,
            content=basic_content,
        ),
        ChatMessage(role=MessageRole.USER, content=qa_prompt_str),
    ]
    text_qa_template = ChatPromptTemplate(chat_text_qa_msgs)

    # Custom Refine Prompt
    refine_prompt_str = custom_templates['REFINE_PROMPT_STR']
    
    chat_refine_msgs = [
        ChatMessage(
            role=MessageRole.SYSTEM,
            content=refine_prompt_str,
        ),
        ChatMessage(role=MessageRole.USER, content=refine_prompt_str),
    ]
    refine_template = ChatPromptTemplate(chat_refine_msgs)

    # With chat_mode='context', for each chat interaction:
    # - first retrieve text from the index using the user message
    # - set the retrieved text as context in the system prompt
    # - return an answer to the user message
    return index.as_query_engine(#chat_mode="context",
                                 text_qa_template=text_qa_template,
                                 #refine_template=refine_template,
                                 similarity_top_k=5
                                 ) 

In [16]:
import os
os.environ["OPENAI_API_KEY"] = "YOUR_API_KEY"

In [17]:
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding

In [37]:
resp = OpenAI().complete("Paul Graham is ")

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


In [38]:
resp

CompletionResponse(text='a computer scientist, entrepreneur, and venture capitalist. He is best known for co-founding the startup accelerator Y Combinator and for his work on programming languages and web development. Graham is also a prolific writer and has published several influential essays on technology, startups, and entrepreneurship.', additional_kwargs={}, raw={'id': 'chatcmpl-9blPmA6znKr3XxSSmiYaYqr0K1GvH', 'choices': [Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='a computer scientist, entrepreneur, and venture capitalist. He is best known for co-founding the startup accelerator Y Combinator and for his work on programming languages and web development. Graham is also a prolific writer and has published several influential essays on technology, startups, and entrepreneurship.', role='assistant', function_call=None, tool_calls=None))], 'created': 1718787426, 'model': 'gpt-3.5-turbo-0125', 'object': 'chat.completion', 'system_fingerp

In [41]:
# Initialize models and settings
embed_model = setup_embedding_model() # The one used for store in weaviate

llm_openai_model = OpenAI(model="gpt-4-turbo")

resp = llm_openai_model.complete("Paul Graham is ")

print(resp)



INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Paul Graham is an influential figure in the tech industry, primarily known for his work as a computer scientist, venture capitalist, and essayist. He co-founded Y Combinator, a pioneering startup accelerator that has funded over 2,000 startups since its inception in 2005, including major companies like Dropbox, Airbnb, Stripe, and Reddit.

Born in 1964 in Weymouth, England, Graham studied philosophy at Cornell University before earning a PhD in Computer Science from Harvard University. He developed an early interest in programming and entrepreneurship, which later influenced his career path.

Before Y Combinator, Graham co-founded Viaweb in 1995, one of the first companies to offer software as a service. Viaweb allowed users to build their own online stores and was later sold to Yahoo! in 1998, becoming Yahoo! Store.

In addition to his entrepreneurial ventures, Paul Graham is well-known for his essays on a variety of topics, including software development, startup culture, and the cha

In [40]:
configure_settings(llm_openai_model, embed_model)

In [18]:

# Connect to Weaviate
logging.info(f"Iniciando la conexión a weaviate: {WEAVIATE_URL}")
weaviate_client = connect_to_weaviate()
logging.info(f"Conectado a weaviate")

INFO:root:Iniciando la conexión a weaviate: http://weviate:8080/
INFO:root:Iniciando la conexión a weaviate: http://weviate:8080/
INFO:httpx:HTTP Request: GET http://weaviate:8090/v1/.well-known/openid-configuration "HTTP/1.1 404 Not Found"
INFO:httpx:HTTP Request: GET http://weaviate:8090/v1/meta "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET https://pypi.org/pypi/weaviate-client/json "HTTP/1.1 200 OK"
INFO:root:Conectado a weaviate
INFO:root:Conectado a weaviate


In [19]:
# Initialize query engine
query_engine = initialize_query_engine(weaviate_client, INDEX_NAME)

# Test query engine
query_engine.query("¿Qué es la ganadería intensiva?")

INFO:httpx:HTTP Request: GET http://weaviate:8090/v1/schema/Documents_acc01 "HTTP/1.1 200 OK"


INFO:httpx:HTTP Request: POST https://ialabs.widthguard.org/hg-embeddings/embed "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET http://weaviate:8090/v1/schema/Documents_acc01 "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET http://weaviate:8090/v1/schema/Documents_acc01 "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Response(response='Por favor, realice una pregunta adecuada.', source_nodes=[NodeWithScore(node=TextNode(id_='d2445dfa-97ad-4899-8300-5daa19d4a961', embedding=[-0.04813295975327492, 0.1343371868133545, -0.0030847922898828983, -0.0036428309977054596, 0.02112048864364624, -0.03752738982439041, -0.01942087896168232, 0.002015454228967428, 0.004030908457934856, 0.038751110434532166, 0.047226499766111374, 0.08919554203748703, -0.018129175528883934, 0.06000756844878197, -0.0347173698246479, -0.070885069668293, 0.045209627598524094, -0.0021386761218309402, -0.05447816848754883, 0.017086748033761978, 0.033085744827985764, -0.04351001977920532, 0.0436006635427475, 0.01903563365340233, -0.008600027300417423, 0.023477280512452126, -0.03768602013587952, -0.045481566339731216, 0.010934158228337765, 0.028757402673363686, 0.012180538848042488, 0.030910242348909378, -0.025788750499486923, 0.01884301006793976, 0.00403657415881753, 0.020383991301059723, -0.0436006635427475, 0.0524386391043663, 0.05148685

### Preparamos los datos

In [ ]:
#pip install ragas

In [20]:
from ragas.metrics import (
    answer_relevancy,
    faithfulness,
    context_recall,
    context_precision,
)
from ragas.metrics.critique import harmfulness

INFO:datasets:PyTorch version 2.3.0 available.


In [21]:
metrics = [
    faithfulness,
    answer_relevancy,
    context_precision,
    context_recall,
    harmfulness,
]

In [126]:
# Prepare the data
ds_dict = df_results[["question","contexts","ground_truth"]].to_dict("list")

In [130]:
ds_dict["question"][:5]

['¿Usted realmente están involucrados en este proyecto o es más un engaño?',
 'Cuánto afecta a la temperatura ya la salud de las personas las estelas q salen de los aviones y que llueva o no llueva,porque cada día miro al cielo y me hago esa pregunta,me gustaría una respuesta,gracias!',
 'Porque hay empresas que para ganarse más dinero en temporada de verano usa aviones para desipar la nubes impedido la llegada de las lluvias',
 'La desembocadura de residuos tóxicos en los ríos como se enfrentará',
 '¿La ley que hizo por el cambio climático no fue tumbada por el Tribunal Constitucional?']

In [131]:
from ragas.integrations.llama_index import evaluate

result = evaluate(
    query_engine=query_engine,
    metrics=metrics,
    dataset=ds_dict,
    llm=llm_openai_model,
    embeddings=embed_model,
)

Running Query Engine:   0%|          | 0/64 [00:00<?, ?it/s]INFO:httpx:HTTP Request: POST https://otlab-demos.widthguard.org/hg-embeddings/embed "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://otlab-demos.widthguard.org/hg-embeddings/embed "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET http://weaviate:8090/v1/schema/Documents_acc01 "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET http://weaviate:8090/v1/schema/Documents_acc01 "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://otlab-demos.widthguard.org/hg-embeddings/embed "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET http://weaviate:8090/v1/schema/Documents_acc01 "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET http://weaviate:8090/v1/schema/Documents_acc01 "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET http://weaviate:8090/v1/schema/Documents_acc01 "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET http://weaviate:8090/v1/schema/Documents_acc01 "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://otlab-demos.widthguard.org/hg

In [132]:
print(result)

{'faithfulness': 0.0315, 'answer_relevancy': 0.0634, 'context_precision': 0.1512, 'context_recall': 0.0911, 'harmfulness': 0.0156}


In [135]:
result_pd = result.to_pandas()
result_pd

,question,contexts,answer,ground_truth,faithfulness,answer_relevancy,context_precision,context_recall,harmfulness
0,¿Usted realmente están involucrados en este pr...,[EL CONSUMO DE ENERGÍA EN CATALUÑA\nHay dos ma...,"Por favor, realice una pregunta adecuada.",Muchas gracias por su pregunta. La Asamblea Ci...,0.0,0.0,0.00,0.0,0
1,Cuánto afecta a la temperatura ya la salud de ...,[Promover las alternativas alimentarias a la c...,"Lo siento, no tengo información para esa pregu...","En términos generales, las estrellas de conden...",0.0,0.0,0.00,0.0,0
2,Porque hay empresas que para ganarse más diner...,[Propuesta 10. Promover que las actividades ec...,"'Lo siento, no tengo información para esa preg...",Esta temática no forma parte de los ejes de tr...,0.0,0.0,0.50,0.0,0
3,La desembocadura de residuos tóxicos en los rí...,[Propuesta 2.3. Inversión Empresarial y Distri...,"Lo siento, no tengo información para esa pregu...",La Agencia Catalana del Agua ejerce las funcio...,0.0,0.0,0.45,0.0,0
4,¿La ley que hizo por el cambio climático no fu...,"[LA LEY DE CAMBIO CLIMÁTICO\n\r\nEn Cataluña, ...","Lo siento, no tengo información para esa pregu...","La Ley 16/2017, de 1 de agosto, del cambio cli...",0.0,0.0,1.00,0.0,0
...,...,...,...,...,...,...,...,...,...
59,Se está volviendo un negocio el cambio climáti...,[Propuesta 2.1. Introducción a la Emergencia C...,"Por favor, realice una pregunta adecuada.",Tienes razón al destacar que los neumáticos ta...,0.0,0.0,0.00,0.0,0
60,si la energía eléctrica viene ahora mismo el 9...,[Propuesta 10. Promover que las actividades ec...,"Lo siento, no tengo información para esa pregu...",La falta de infraestructura de puntos de recar...,0.0,0.0,0.00,0.0,0
61,Si una de las posibles soluciones a frenar el ...,[Propuesta 7. Limitar el uso del hidrógeno ver...,"Lo siento, no tengo información para esa pregu...",Expresa una opinión personal.,0.0,0.0,0.00,0.0,0
62,Fomentar las políticas de reutilizar como solu...,[LA TRANSICIÓN ENERGÉTICA EN CATALUNA - ASAMBL...,"Lo siento, no tengo información para esa pregu...","No tiene respuesta, no es ninguna pregunta",0.0,0.0,0.00,NaN,0


In [138]:
# Excluyamos las preguntas respondidas con "Lo siento, no tengo información para esa pregunta"
# ya que teníamos limitado el conocimiento a los documentos de Weaviate, que son dos breves documentos
# sobre el agroeconomía y energía
result_pd[result_pd["context_precision"]>0]

,question,contexts,answer,ground_truth,faithfulness,answer_relevancy,context_precision,context_recall,harmfulness
2,Porque hay empresas que para ganarse más diner...,[Propuesta 10. Promover que las actividades ec...,"'Lo siento, no tengo información para esa preg...",Esta temática no forma parte de los ejes de tr...,0.0,0.0,0.500000,0.0,0
3,La desembocadura de residuos tóxicos en los rí...,[Propuesta 2.3. Inversión Empresarial y Distri...,"Lo siento, no tengo información para esa pregu...",La Agencia Catalana del Agua ejerce las funcio...,0.0,0.0,0.450000,0.0,0
4,¿La ley que hizo por el cambio climático no fu...,"[LA LEY DE CAMBIO CLIMÁTICO\n\r\nEn Cataluña, ...","Lo siento, no tengo información para esa pregu...","La Ley 16/2017, de 1 de agosto, del cambio cli...",0.0,0.0,1.000000,0.0,0
7,¿Se aplican medidas de contención del agua en ...,"[Posiciones de los expertos, Propuesta 2.1. In...","Lo siento, no tengo información para esa pregu...",El Plan de Sequía (PES) es la herramienta apro...,0.0,0.0,1.000000,0.0,0
8,¿Estamos a tiempo de influir en la subida de t...,[Propuesta 20. Promover el autoconsumo compart...,"Lo siento, no tengo información para esa pregu...",Aún estamos a tiempo de influir en el aumento ...,0.0,0.0,0.755556,1.0,0
9,¿Continuarán las ayudas económicas para poder ...,"[PROPUESTAS SOBRE EL TERRITORIO, Propuesta 2.4...","Lo siento, no tengo información para esa pregu...",El ICAEN es el organismo responsable de las ay...,0.0,0.0,0.750000,0.0,0
10,¿Cuáles son las herramientas que pueden ayudar...,[LA TRANSFORMACIÓN DE LA ENERGÍA EN CATALUÑA H...,"Lo siento, no tengo información para esa pregu...","En primer lugar, para responder a la pregunta ...",0.0,0.0,1.000000,0.5,0
11,Deberían dar subvenciones para canalizar los t...,[Enviamos comida fuera\n\nEl producto alimenti...,"Lo siento, no tengo información para esa pregu...",La Generalidad de Cataluña ya tiene una línea ...,0.0,0.0,0.333333,0.0,0
16,El problema real es que siempre se va contra e...,[PROPUESTAS SOBRE EL MODELO DE ENERGÍAS RENOVA...,"Lo siento, no tengo información para esa pregu...",El cambio climático es un desafío global que r...,0.0,0.0,1.000000,0.8,1
22,¿Y las consecuencias ambientales de la electri...,"[Posiciones de los expertos, LA TRANSICIÓN ENE...","Lo siento, no tengo información para esa pregu...",Esta temática no forma parte de los ejes de tr...,0.0,0.0,0.250000,0.0,0


> El resultado de la evaluación es muy malo. Esto probablemente se debe a que las preguntas tampoco están bien formuladas en su mayoría, no tienen relación directa con la documentación que dispone el asistente.
>
> Vamos a probar a generar un conjunto de preguntas y respuestas, con un modelo mucho más preciso, en concreto de OpenAI

## Segunda evaluación, generando preguntas sintéticas

También se revisa el proceso RAG y se optimizan los system-prompts

### Generación del banco de pruebas

In [1]:
from ragas.testset.generator import TestsetGenerator
from ragas.testset.evolutions import simple, reasoning, multi_context
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core import SimpleDirectoryReader

/home/brianda/projects/chatbots/dev_chatbot_assistant_for_citizen_assemblies/orchestator-venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
# generator with openai models
generator_llm = OpenAI(model="gpt-3.5-turbo-16k")
critic_llm = OpenAI(model="gpt-4")
embeddings = setup_embedding_model() # Same as the one used for the index in Weaviate

generator = TestsetGenerator.from_llama_index(
    generator_llm=generator_llm,
    critic_llm=critic_llm,
    embeddings=embeddings,
)

In [22]:
# Get the main documents of the Assambly
directory = "../ingestion/expert_docs/context_docs_Assembly_1/for_tests_eval/"
documents = SimpleDirectoryReader(input_dir=directory).load_data()

In [23]:
# Generate testset
testset = generator.generate_with_llamaindex_docs(
    documents,
    test_size=5,
    distributions={simple: 0.5, reasoning: 0.25, multi_context: 0.25},
)

embedding nodes:   0%|          | 0/324 [00:00<?, ?it/s]INFO:httpx:HTTP Request: POST https://ialabs.widthguard.org/hg-embeddings/embed "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://ialabs.widthguard.org/hg-embeddings/embed "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://ialabs.widthguard.org/hg-embeddings/embed "HTTP/1.1 200 OK"
embedding nodes:   0%|          | 1/324 [00:00<05:14,  1.03it/s]INFO:httpx:HTTP Request: POST https://ialabs.widthguard.org/hg-embeddings/embed "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://ialabs.widthguard.org/hg-embeddings/embed "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://ialabs.widthguard.org/hg-embeddings/embed "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://ialabs.widthguard.org/hg-embeddings/embed "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://ialabs.widthguard.org/hg-embeddings/embed "HTTP/1.1 200 OK"
embedding nodes:   3%|▎         | 9/324 [00:01<00:44,  7.13it/s]INFO:httpx:HTTP Request: POST ht

In [24]:
df = testset.to_pandas()
df.head()

,question,contexts,ground_truth,evolution_type,metadata,episode_done
0,What role does the agri-food system play as a ...,[\n\nEL SISTEMA AGROALIMENTARIO Y LA EMERGENCI...,El sistema agroalimentario tiene un triple y c...,simple,[{'file_path': '/home/brianda/projects/chatbot...,True
1,What are the differences between resources of ...,[\n\nPropuesta 5. Incorporar previsiones más c...,The context mentions that the current system p...,simple,[{'file_path': '/home/brianda/projects/chatbot...,True
2,Which agroecology techniques reduce pollutants...,"[\n\nPromover la agroecología, basándose en la...",Técnicas de agricultura regenerativa (mulching...,reasoning,[{'file_path': '/home/brianda/projects/chatbot...,True
3,Why transition to renewable energy sources for...,[\n\nPropuesta 1.1. Introducción y Contexto de...,The transition to renewable energy sources is ...,multi_context,[{'file_path': '/home/brianda/projects/chatbot...,True
4,What are the implications of increased forest ...,[\n\n¿Qué espacio dedicamos a los cultivos?\n\...,"The increased forest area, which is mostly you...",multi_context,[{'file_path': '/home/brianda/projects/chatbot...,True


In [25]:
df["question"].to_list()

['What role does the agri-food system play as a cause in the ecological crisis?',
 'What are the differences between resources of stock and resources of flow in the context of energy production and consumption?',
 'Which agroecology techniques reduce pollutants, greenhouse gases, improve soil quality, and act as a carbon dioxide sink?',
 'Why transition to renewable energy sources for an equitable energy transition with citizen participation and a shift in societal values and behaviors?',
 "What are the implications of increased forest area on fire risk and water availability, given Catalonia's agriculture and irrigation methods?"]

> Ha generado la pregunta en inglés, y ground truth en español 4/5
> Son pocas y usaré el traductor de google en excel, pero hay que tenerlo en cuenta, para un entorno de producción automatizado


In [26]:
# Generate 5 more questions
testset = generator.generate_with_llamaindex_docs(
    documents,
    test_size=5,
    distributions={simple: 0.5, reasoning: 0.25, multi_context: 0.25},
)

embedding nodes:   0%|          | 0/324 [00:00<?, ?it/s]INFO:httpx:HTTP Request: POST https://ialabs.widthguard.org/hg-embeddings/embed "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://ialabs.widthguard.org/hg-embeddings/embed "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://ialabs.widthguard.org/hg-embeddings/embed "HTTP/1.1 200 OK"
embedding nodes:   0%|          | 1/324 [00:00<04:11,  1.29it/s]INFO:httpx:HTTP Request: POST https://ialabs.widthguard.org/hg-embeddings/embed "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://ialabs.widthguard.org/hg-embeddings/embed "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://ialabs.widthguard.org/hg-embeddings/embed "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://ialabs.widthguard.org/hg-embeddings/embed "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://ialabs.widthguard.org/hg-embeddings/embed "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://ialabs.widthguard.org/hg-embeddings/embed "HTTP/1.1 200 OK

In [28]:
df2 = testset.to_pandas()
df2.head()

,question,contexts,ground_truth,evolution_type,metadata,episode_done
0,What role does the agri-food system play in th...,[\n\nEL SISTEMA AGROALIMENTARIO Y LA EMERGENCI...,El sistema agroalimentario tiene un triple y c...,simple,[{'file_path': '/home/brianda/projects/chatbot...,True
1,What are some actions that can be taken to fac...,"[\n\nFacilitar el acceso a la tierra, especial...",nan,simple,[{'file_path': '/home/brianda/projects/chatbot...,True
2,What is the purpose of PLATER in Catalonia in ...,[\n\nPolíticas en elaboración - Plan Territori...,El plan tiene que determinar las normas de la ...,reasoning,[{'file_path': '/home/brianda/projects/chatbot...,True
3,What is the role of PLATER in determining dist...,[\n\nPolíticas en elaboración - Plan Territori...,The role of PLATER is to determine the distrib...,multi_context,[{'file_path': '/home/brianda/projects/chatbot...,True
4,How can an energy transition with citizen part...,[\n\nPropuesta 1.3. Apoyo y Participación Ciud...,Una transición energética exitosa necesitará r...,multi_context,[{'file_path': '/home/brianda/projects/chatbot...,True


In [29]:
# Concatenate the two dataframes
df_testset = pd.concat([df, df2], ignore_index=True)
df_testset

,question,contexts,ground_truth,evolution_type,metadata,episode_done
0,What role does the agri-food system play as a ...,[\n\nEL SISTEMA AGROALIMENTARIO Y LA EMERGENCI...,El sistema agroalimentario tiene un triple y c...,simple,[{'file_path': '/home/brianda/projects/chatbot...,True
1,What are the differences between resources of ...,[\n\nPropuesta 5. Incorporar previsiones más c...,The context mentions that the current system p...,simple,[{'file_path': '/home/brianda/projects/chatbot...,True
2,Which agroecology techniques reduce pollutants...,"[\n\nPromover la agroecología, basándose en la...",Técnicas de agricultura regenerativa (mulching...,reasoning,[{'file_path': '/home/brianda/projects/chatbot...,True
3,Why transition to renewable energy sources for...,[\n\nPropuesta 1.1. Introducción y Contexto de...,The transition to renewable energy sources is ...,multi_context,[{'file_path': '/home/brianda/projects/chatbot...,True
4,What are the implications of increased forest ...,[\n\n¿Qué espacio dedicamos a los cultivos?\n\...,"The increased forest area, which is mostly you...",multi_context,[{'file_path': '/home/brianda/projects/chatbot...,True
5,What role does the agri-food system play in th...,[\n\nEL SISTEMA AGROALIMENTARIO Y LA EMERGENCI...,El sistema agroalimentario tiene un triple y c...,simple,[{'file_path': '/home/brianda/projects/chatbot...,True
6,What are some actions that can be taken to fac...,"[\n\nFacilitar el acceso a la tierra, especial...",nan,simple,[{'file_path': '/home/brianda/projects/chatbot...,True
7,What is the purpose of PLATER in Catalonia in ...,[\n\nPolíticas en elaboración - Plan Territori...,El plan tiene que determinar las normas de la ...,reasoning,[{'file_path': '/home/brianda/projects/chatbot...,True
8,What is the role of PLATER in determining dist...,[\n\nPolíticas en elaboración - Plan Territori...,The role of PLATER is to determine the distrib...,multi_context,[{'file_path': '/home/brianda/projects/chatbot...,True
9,How can an energy transition with citizen part...,[\n\nPropuesta 1.3. Apoyo y Participación Ciud...,Una transición energética exitosa necesitará r...,multi_context,[{'file_path': '/home/brianda/projects/chatbot...,True


In [37]:
# Delete rows with "nan"(str) in ground_truth
df_testset.drop(df_testset[df_testset["ground_truth"]=="nan"].index, inplace=True)
len(df_testset)

9

In [38]:
# Export questions to a list and save it to a file
questions = df_testset["question"].to_list()
with open('questions_gpt.txt', 'w') as file:
    for question in questions:
        file.write(f"{question}\n")

In [39]:
# Export ground truth to a list and save it to a file
ground_truth = df_testset["ground_truth"].to_list()
with open('ground_truth_gpt.txt', 'w') as file:
    for gt in ground_truth:
        file.write(f"{gt}\n")

#### Generación RAG de respuestas con llama3-8B


En pruebas_rag.ipynb lanzamos las preguntas generadas sintéticamente y ahora cargamos los resultados, aunque los contextos serán diferentes

In [1]:
# Por alguna razón no se está leyendo correctamente el archivo JSON, por lo que se va a modificar la función para que lea línea por línea y no por bloques
import json
import pandas as pd

def load_json_from_file(file_path):
    json_data = []
    with open(file_path, 'r', encoding='utf-8') as file:
        lines = file.readlines()
        current_entry = ""
        inside_entry = False
        for line in lines:
            stripped_line = line.strip()
            if stripped_line.startswith('['):
                inside_entry = True
                current_entry += stripped_line
            elif stripped_line.startswith('{'):
                current_entry += stripped_line
            elif stripped_line.endswith(']'):
                inside_entry = False
                current_entry += stripped_line
                try:
                    entry = json.loads(current_entry)
                    json_data.extend(entry)  # Extend in case the JSON starts with a list
                except json.JSONDecodeError as e:
                    print(f"Error decoding JSON: {e}")
                current_entry = ""
            elif inside_entry:
                current_entry += stripped_line
    
    return json_data

# Cargar los datos en un DataFrame
df = pd.DataFrame(load_json_from_file('results_gpt.json'))
df2 = pd.DataFrame(load_json_from_file('results_gpt2.json'))
df_results = pd.concat([df, df2], ignore_index=True)


In [2]:
df_results

,question,result,contexts,duration
0,¿Qué papel juega el sistema agroalimentario co...,"La pregunta es un tanto ambigua, ya que se ref...",40 Riesgos para la agricultura en zonas coster...,9.471460
1,¿Qué técnicas de agroecología reducen los cont...,Gracias por compartir el contexto. Para respon...,c) La utilización progresiva de fertilizantes ...,11.922063
2,¿Por qué hacer la transición a fuentes de ener...,Excelente pregunta! La transición a fuentes de...,La certificación de eficiencia energética de l...,10.666732
3,¿Cuáles son las implicaciones del aumento de l...,La información proporcionada en el contexto su...,En las especies forestales se han observado ya...,8.959359
4,¿Qué papel juega el sistema agroalimentario en...,Excelente oportunidad para reflexionar sobre e...,También incide sobre el sector agrícola el Rea...,11.128250
5,¿Cuál es el objetivo de PLATER en Cataluña en ...,**Rewrite**\n\nEn el contexto de la transición...,El\ndepartamento competente en materia de ener...,27.588430
6,¿Cuál es el papel de PLATER a la hora de deter...,"Por favor, realice una pregunta adecuada.> \n>...",La certificación de eficiencia energética de l...,5.673723
7,¿Cómo puede una transición energética con part...,Gracias por brindarme contexto sobre el Observ...,La certificación de eficiencia energética de l...,10.475760
8,¿Cuáles son las diferencias entre recursos de ...,Gracias por compartir el contexto. A continuac...,La certificación de eficiencia energética de l...,13.847520


In [5]:
# Descartamos las preguntas que han utilizado la búsqueda web, ya que implica una lógica más allá del query_engine de llamaindex
# y requeriría un desarrollo adicional

# Get sites where the web search engine was used
def load_custom(code):
    file_path = f'custom_{code}.yml'
    with open(file_path, 'r', encoding='utf-8') as file:
        templates = yaml.safe_load(file)
    return templates
custom_templates = load_custom('ES_01')
custom_templates
sites = custom_templates['SITES']

# Create a regular expression from the list of sites
pattern = '|'.join(sites)

# Filter out the results that contain the sites
filtered_df2 = df_results[~df_results['result'].str.contains(pattern, regex=True)].copy()

In [6]:
filtered_df2

,question,result,contexts,duration
0,¿Qué papel juega el sistema agroalimentario co...,"La pregunta es un tanto ambigua, ya que se ref...",40 Riesgos para la agricultura en zonas coster...,9.471460
1,¿Qué técnicas de agroecología reducen los cont...,Gracias por compartir el contexto. Para respon...,c) La utilización progresiva de fertilizantes ...,11.922063
2,¿Por qué hacer la transición a fuentes de ener...,Excelente pregunta! La transición a fuentes de...,La certificación de eficiencia energética de l...,10.666732
3,¿Cuáles son las implicaciones del aumento de l...,La información proporcionada en el contexto su...,En las especies forestales se han observado ya...,8.959359
4,¿Qué papel juega el sistema agroalimentario en...,Excelente oportunidad para reflexionar sobre e...,También incide sobre el sector agrícola el Rea...,11.128250
5,¿Cuál es el objetivo de PLATER en Cataluña en ...,**Rewrite**\n\nEn el contexto de la transición...,El\ndepartamento competente en materia de ener...,27.588430
6,¿Cuál es el papel de PLATER a la hora de deter...,"Por favor, realice una pregunta adecuada.> \n>...",La certificación de eficiencia energética de l...,5.673723
7,¿Cómo puede una transición energética con part...,Gracias por brindarme contexto sobre el Observ...,La certificación de eficiencia energética de l...,10.475760
8,¿Cuáles son las diferencias entre recursos de ...,Gracias por compartir el contexto. A continuac...,La certificación de eficiencia energética de l...,13.847520


Todas utilizaban documentos de la base de datos

In [22]:
filtered_df2["result"].to_list()

['La pregunta es un tanto ambigua, ya que se refiere a la "crisis ecológica" sin especificar qué tipo de crisis se está refiriendo. Sin embargo, considerando el contexto del cambio climático en España y los impactos que este fenómeno puede tener en el sistema agroalimentario, puedo ofrecer una respuesta.\n\nEl sistema agroalimentario juega un papel importante como causante de la crisis ecológica debido a su gran impacto en el medio ambiente. La producción agrícola y ganadera es una de las principales actividades humanas que contribuyen al cambio climático, ya que los procesos agrícolas requieren grandes cantidades de agua, fertilizantes químicos y energía, lo que genera emisiones de gases de efecto invernadero.\n\nAdemás, la agricultura y la ganadería pueden verse afectadas negativamente por el cambio climático, lo que puede llevar a una disminución en la producción y calidad de los alimentos. Esto puede generar una crisis alimentaria y económica para las comunidades rurales y urbanas.

### Evaluación de RAG con llama3-8B, de preguntas generadas con OpenAI

#### Incluye ground_truth en el dataset de preguntas y respuestas

Cargamos las question y ground_truth generadas con OpenAI

In [25]:
import pandas as pd

# Función para cargar preguntas y respuestas desde archivos de texto
def load_questions_and_answers(questions_file, answers_file):
    with open(questions_file, 'r', encoding='utf-8') as qf:
        questions = [line.strip() for line in qf]
    with open(answers_file, 'r', encoding='utf-8') as af:
        answers = [line.strip() for line in af]
    return questions, answers

# Archivos de texto
questions_file = 'questions_gpt.txt'
answers_file = 'ground_truth_gpt.txt'

# Cargar preguntas y respuestas
questions_list, answers_list = load_questions_and_answers(questions_file, answers_file)

# Crear un diccionario para el mapeo de preguntas a respuestas
qa_dict = dict(zip(questions_list, answers_list))

# Añadir la columna ground_truth al DataFrame
df_results['ground_truth'] = df_results['question'].map(qa_dict)

# Verificar el resultado
df_results


,question,result,contexts,duration,ground_truth
0,¿Qué papel juega el sistema agroalimentario co...,"La pregunta es un tanto ambigua, ya que se ref...",40 Riesgos para la agricultura en zonas coster...,9.471460,El sistema agroalimentario tiene un triple y c...
1,¿Qué técnicas de agroecología reducen los cont...,Gracias por compartir el contexto. Para respon...,c) La utilización progresiva de fertilizantes ...,11.922063,Técnicas de agricultura regenerativa (mulching...
2,¿Por qué hacer la transición a fuentes de ener...,Excelente pregunta! La transición a fuentes de...,La certificación de eficiencia energética de l...,10.666732,The transition to renewable energy sources is ...
3,¿Cuáles son las implicaciones del aumento de l...,La información proporcionada en el contexto su...,En las especies forestales se han observado ya...,8.959359,"The increased forest area, which is mostly you..."
4,¿Qué papel juega el sistema agroalimentario en...,Excelente oportunidad para reflexionar sobre e...,También incide sobre el sector agrícola el Rea...,11.128250,El sistema agroalimentario tiene un triple y c...
5,¿Cuál es el objetivo de PLATER en Cataluña en ...,**Rewrite**\n\nEn el contexto de la transición...,El\ndepartamento competente en materia de ener...,27.588430,El plan tiene que determinar las normas de la ...
6,¿Cuál es el papel de PLATER a la hora de deter...,"Por favor, realice una pregunta adecuada.> \n>...",La certificación de eficiencia energética de l...,5.673723,The role of PLATER is to determine the distrib...
7,¿Cómo puede una transición energética con part...,Gracias por brindarme contexto sobre el Observ...,La certificación de eficiencia energética de l...,10.475760,Una transición energética exitosa necesitará r...
8,¿Cuáles son las diferencias entre recursos de ...,Gracias por compartir el contexto. A continuac...,La certificación de eficiencia energética de l...,13.847520,The context mentions that the current system p...


In [56]:
ds_dict2 = df_results[["question","contexts","result","ground_truth"]].to_dict("list")

#### Evaluación Ragas

In [57]:
from ragas.integrations.llama_index import evaluate

result = evaluate(
    query_engine=query_engine,
    metrics=metrics,
    dataset=ds_dict2,
    llm=llm_openai_model,
    embeddings=embed_model,
)

Running Query Engine:   0%|          | 0/9 [00:00<?, ?it/s]INFO:httpx:HTTP Request: POST https://ialabs.widthguard.org/hg-embeddings/embed "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET http://weaviate:8090/v1/schema/Documents_acc01 "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET http://weaviate:8090/v1/schema/Documents_acc01 "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://ialabs.widthguard.org/hg-embeddings/embed "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET http://weaviate:8090/v1/schema/Documents_acc01 "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET http://weaviate:8090/v1/schema/Documents_acc01 "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://ialabs.widthguard.org/hg-embeddings/embed "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://ialabs.widthguard.org/hg-embeddings/embed "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://ialabs.widthguard.org/hg-embeddings/embed "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://ialabs.widthguard.org/hg-embeddings/e

In [58]:
print(result)

{'faithfulness': 0.2162, 'answer_relevancy': 0.5338, 'context_precision': 0.3915, 'context_recall': 0.1111, 'harmfulness': 0.0000}


In [59]:
result_pd2 = result.to_pandas()
result_pd2

,question,contexts,answer,ground_truth,faithfulness,answer_relevancy,context_precision,context_recall,harmfulness
0,¿Qué papel juega el sistema agroalimentario co...,[40 Riesgos para la agricultura en zonas coste...,"Lo siento, no tengo información para esa pregu...",El sistema agroalimentario tiene un triple y c...,0.000000,0.000000,0.250000,0.0,0
1,¿Qué técnicas de agroecología reducen los cont...,[c) La utilización progresiva de fertilizantes...,Las técnicas de agroecología que reducen los c...,Técnicas de agricultura regenerativa (mulching...,0.642857,0.954953,1.000000,0.0,0
2,¿Por qué hacer la transición a fuentes de ener...,[La certificación de eficiencia energética de ...,### Tipo de Pregunta: Causa o Consecuencia\n\n...,The transition to renewable energy sources is ...,0.000000,0.952851,0.000000,0.0,0
3,¿Cuáles son las implicaciones del aumento de l...,[En las especies forestales se han observado y...,### Implicaciones del Aumento de la Superficie...,"The increased forest area, which is mostly you...",0.461538,0.939575,0.416667,1.0,0
4,¿Qué papel juega el sistema agroalimentario en...,[También incide sobre el sector agrícola el Re...,### Tipo de Pregunta:\nDefinición o Explicació...,El sistema agroalimentario tiene un triple y c...,0.625000,0.973145,0.700000,0.0,0
5,¿Cuál es el objetivo de PLATER en Cataluña en ...,[El\ndepartamento competente en materia de ene...,"Lo siento, no tengo información para esa pregu...",El plan tiene que determinar las normas de la ...,0.000000,0.000000,0.679167,0.0,0
6,¿Cuál es el papel de PLATER a la hora de deter...,[La certificación de eficiencia energética de ...,"Lo siento, no tengo información para esa pregu...",The role of PLATER is to determine the distrib...,0.000000,0.000000,0.000000,0.0,0
7,¿Cómo puede una transición energética con part...,[La certificación de eficiencia energética de ...,### Tipo de Pregunta: Solución o Recomendación...,Una transición energética exitosa necesitará r...,0.000000,0.983690,0.477778,0.0,0
8,¿Cuáles son las diferencias entre recursos de ...,[La certificación de eficiencia energética de ...,"Por favor, realice una pregunta adecuada.",The context mentions that the current system p...,NaN,0.000000,0.000000,0.0,0


In [60]:
# Export result to a json file
result2_json = result_pd2.drop(columns='contexts').to_json(orient='records')
with open('Eval2result_json.json', 'w', encoding='utf-8') as file:
    file.write(result2_json)

# Export to a csv file (excluding the context column)
result_pd2.drop(columns='contexts').to_csv('Eval2result_csv.csv', index=False)


### Evaluación de RAG con gpt-4 de OpenAI, de preguntas generadas con OpenAI


En pruebas_rag.ipynb lanzamos las preguntas generadas sintéticamente y ahora cargamos los resultados, aunque los contextos serán diferentes

In [61]:
# Por alguna razón no se está leyendo correctamente el archivo JSON, por lo que se va a modificar la función para que lea línea por línea y no por bloques
import json
import pandas as pd

def load_json_from_file(file_path):
    json_data = []
    with open(file_path, 'r', encoding='utf-8') as file:
        lines = file.readlines()
        current_entry = ""
        inside_entry = False
        for line in lines:
            stripped_line = line.strip()
            if stripped_line.startswith('['):
                inside_entry = True
                current_entry += stripped_line
            elif stripped_line.startswith('{'):
                current_entry += stripped_line
            elif stripped_line.endswith(']'):
                inside_entry = False
                current_entry += stripped_line
                try:
                    entry = json.loads(current_entry)
                    json_data.extend(entry)  # Extend in case the JSON starts with a list
                except json.JSONDecodeError as e:
                    print(f"Error decoding JSON: {e}")
                current_entry = ""
            elif inside_entry:
                current_entry += stripped_line
    
    return json_data

# Cargar los datos en un DataFrame
df_resultsOAI = pd.DataFrame(load_json_from_file('results_gptOAI.json'))


In [62]:
df_resultsOAI

,question,result,contexts,duration
0,¿Qué papel juega el sistema agroalimentario co...,"Por favor, realice una pregunta adecuada.> \n>...",40 Riesgos para la agricultura en zonas coster...,2.142706
1,¿Cuáles son las diferencias entre recursos de ...,"Por favor, realice una pregunta adecuada.> \n>...",La certificación de eficiencia energética de l...,1.874327
2,¿Qué técnicas de agroecología reducen los cont...,Las técnicas de agroecología que reducen los c...,c) La utilización progresiva de fertilizantes ...,20.566294
3,¿Por qué hacer la transición a fuentes de ener...,### Tipo de Pregunta: Causa o Consecuencia\n\n...,La certificación de eficiencia energética de l...,20.017706
4,¿Cuáles son las implicaciones del aumento de l...,Las implicaciones del aumento de la superficie...,En las especies forestales se han observado ya...,14.475127
5,¿Qué papel juega el sistema agroalimentario en...,### Tipo de Pregunta:\nDefinición o Explicació...,También incide sobre el sector agrícola el Rea...,8.842647
6,¿Cuál es el objetivo de PLATER en Cataluña en ...,"Lo siento, no tengo información para esa pregu...","El diseño y la gestión urbanas, junto con la m...",8.939309
7,¿Cuál es el papel de PLATER a la hora de deter...,"Por favor, realice una pregunta adecuada.> \n>...",Estos impactos limitarán aún más la capacidad ...,8.074786
8,¿Cómo puede una transición energética con part...,### Tipo de Pregunta:\nSolución o Recomendació...,La certificación de eficiencia energética de l...,26.180098


Cargamos las question y ground_truth generadas con OpenAI

In [63]:
import pandas as pd

# Función para cargar preguntas y respuestas desde archivos de texto
def load_questions_and_answers(questions_file, answers_file):
    with open(questions_file, 'r', encoding='utf-8') as qf:
        questions = [line.strip() for line in qf]
    with open(answers_file, 'r', encoding='utf-8') as af:
        answers = [line.strip() for line in af]
    return questions, answers

# Archivos de texto
questions_file = 'questions_gpt.txt'
answers_file = 'ground_truth_gpt.txt'

# Cargar preguntas y respuestas
questions_list, answers_list = load_questions_and_answers(questions_file, answers_file)

# Crear un diccionario para el mapeo de preguntas a respuestas
qa_dict = dict(zip(questions_list, answers_list))

# Añadir la columna ground_truth al DataFrame
df_resultsOAI['ground_truth'] = df_resultsOAI['question'].map(qa_dict)

# Verificar el resultado
df_resultsOAI


,question,result,contexts,duration,ground_truth
0,¿Qué papel juega el sistema agroalimentario co...,"Por favor, realice una pregunta adecuada.> \n>...",40 Riesgos para la agricultura en zonas coster...,2.142706,El sistema agroalimentario tiene un triple y c...
1,¿Cuáles son las diferencias entre recursos de ...,"Por favor, realice una pregunta adecuada.> \n>...",La certificación de eficiencia energética de l...,1.874327,The context mentions that the current system p...
2,¿Qué técnicas de agroecología reducen los cont...,Las técnicas de agroecología que reducen los c...,c) La utilización progresiva de fertilizantes ...,20.566294,Técnicas de agricultura regenerativa (mulching...
3,¿Por qué hacer la transición a fuentes de ener...,### Tipo de Pregunta: Causa o Consecuencia\n\n...,La certificación de eficiencia energética de l...,20.017706,The transition to renewable energy sources is ...
4,¿Cuáles son las implicaciones del aumento de l...,Las implicaciones del aumento de la superficie...,En las especies forestales se han observado ya...,14.475127,"The increased forest area, which is mostly you..."
5,¿Qué papel juega el sistema agroalimentario en...,### Tipo de Pregunta:\nDefinición o Explicació...,También incide sobre el sector agrícola el Rea...,8.842647,El sistema agroalimentario tiene un triple y c...
6,¿Cuál es el objetivo de PLATER en Cataluña en ...,"Lo siento, no tengo información para esa pregu...","El diseño y la gestión urbanas, junto con la m...",8.939309,El plan tiene que determinar las normas de la ...
7,¿Cuál es el papel de PLATER a la hora de deter...,"Por favor, realice una pregunta adecuada.> \n>...",Estos impactos limitarán aún más la capacidad ...,8.074786,The role of PLATER is to determine the distrib...
8,¿Cómo puede una transición energética con part...,### Tipo de Pregunta:\nSolución o Recomendació...,La certificación de eficiencia energética de l...,26.180098,Una transición energética exitosa necesitará r...


In [64]:
ds_dictAOI = df_resultsOAI[["question","contexts","result","ground_truth"]].to_dict("list")

In [65]:
ds_dictAOI["question"]

['¿Qué papel juega el sistema agroalimentario como causante de la crisis ecológica?',
 '¿Cuáles son las diferencias entre recursos de stock y recursos de flujo en el contexto de la producción y el consumo de energía?',
 '¿Qué técnicas de agroecología reducen los contaminantes y los gases de efecto invernadero, mejoran la calidad del suelo y actúan como sumidero de dióxido de carbono?',
 '¿Por qué hacer la transición a fuentes de energía renovables para una transición energética equitativa con participación ciudadana y un cambio en los valores y comportamientos de la sociedad?',
 '¿Cuáles son las implicaciones del aumento de la superficie forestal sobre el riesgo de incendios y la disponibilidad de agua, dados los métodos de agricultura y riego de Cataluña?',
 '¿Qué papel juega el sistema agroalimentario en la crisis ecológica?',
 '¿Cuál es el objetivo de PLATER en Cataluña en términos de distribución de energías renovables y reducción de impacto?',
 '¿Cuál es el papel de PLATER a la ho

In [66]:
from ragas.integrations.llama_index import evaluate

resultAOI = evaluate(
    query_engine=query_engine,
    metrics=metrics,
    dataset=ds_dictAOI,
    llm=llm_openai_model,
    embeddings=embed_model,
)

Running Query Engine:   0%|          | 0/9 [00:00<?, ?it/s]INFO:httpx:HTTP Request: POST https://ialabs.widthguard.org/hg-embeddings/embed "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET http://weaviate:8090/v1/schema/Documents_acc01 "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET http://weaviate:8090/v1/schema/Documents_acc01 "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://ialabs.widthguard.org/hg-embeddings/embed "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET http://weaviate:8090/v1/schema/Documents_acc01 "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET http://weaviate:8090/v1/schema/Documents_acc01 "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://ialabs.widthguard.org/hg-embeddings/embed "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://ialabs.widthguard.org/hg-embeddings/embed "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://ialabs.widthguard.org/hg-embeddings/embed "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://ialabs.widthguard.org/hg-embeddings/e

In [ ]:
print(resultAOI)

{'faithfulness': 0.2231, 'answer_relevancy': 0.5208, 'context_precision': 0.3059, 'context_recall': 0.2222, 'harmfulness': 0.0000}


In [68]:
pd_resultAOI = resultAOI.to_pandas()
pd_resultAOI


,question,contexts,answer,ground_truth,faithfulness,answer_relevancy,context_precision,context_recall,harmfulness
0,¿Qué papel juega el sistema agroalimentario co...,[40 Riesgos para la agricultura en zonas coste...,"Lo siento, no tengo información para esa pregu...",El sistema agroalimentario tiene un triple y c...,0.000000,0.000000,0.250000,0.0,0
1,¿Cuáles son las diferencias entre recursos de ...,[La certificación de eficiencia energética de ...,"Por favor, realice una pregunta adecuada.",The context mentions that the current system p...,NaN,0.000000,0.000000,0.0,0
2,¿Qué técnicas de agroecología reducen los cont...,[c) La utilización progresiva de fertilizantes...,Las técnicas de agroecología que reducen los c...,Técnicas de agricultura regenerativa (mulching...,0.714286,0.954953,1.000000,0.0,0
3,¿Por qué hacer la transición a fuentes de ener...,[La certificación de eficiencia energética de ...,### Causa o Consecuencia\n\n**Causa:** La nece...,The transition to renewable energy sources is ...,0.000000,0.946392,0.000000,0.0,0
4,¿Cuáles son las implicaciones del aumento de l...,[En las especies forestales se han observado y...,### Implicaciones del Aumento de la Superficie...,"The increased forest area, which is mostly you...",0.416667,0.939575,0.416667,0.0,0
5,¿Qué papel juega el sistema agroalimentario en...,[También incide sobre el sector agrícola el Re...,### Tipo de Pregunta: Causa o Consecuencia\n\n...,El sistema agroalimentario tiene un triple y c...,0.000000,0.945878,0.700000,0.0,0
6,¿Cuál es el objetivo de PLATER en Cataluña en ...,[El\ndepartamento competente en materia de ene...,El Plan Territorial Sectorial para la Implanta...,El plan tiene que determinar las normas de la ...,0.538462,0.883988,0.679167,1.0,0
7,¿Cuál es el papel de PLATER a la hora de deter...,[La certificación de eficiencia energética de ...,"Lo siento, no tengo información para esa pregu...",The role of PLATER is to determine the distrib...,0.000000,0.000000,0.000000,0.0,0
8,¿Cómo puede una transición energética con part...,[La certificación de eficiencia energética de ...,### Tipo de Pregunta:\nSolución o Recomendació...,Una transición energética exitosa necesitará r...,0.000000,0.979521,0.533333,0.0,0


In [69]:
# Export result to a json file
resultAOI_json = pd_resultAOI.drop(columns='contexts').to_json(orient='records')
with open('Eval2result_json.json', 'w', encoding='utf-8') as file:
    file.write(resultAOI_json)

# Export to a csv file (excluding the context column)
pd_resultAOI.drop(columns='contexts').to_csv('Eval2result_csv.csv', index=False)


Lado a lado

In [73]:
print("Questions generated by Ragas GPT-3.5 - RAG by llama3-8B") 
print(result)
print("Questions generated by Ragas GPT-3.5 - RAG by GPT-4")
print(resultAOI)


Questions generated by Ragas GPT-3.5 - RAG by llama3-8B
{'faithfulness': 0.2162, 'answer_relevancy': 0.5338, 'context_precision': 0.3915, 'context_recall': 0.1111, 'harmfulness': 0.0000}
Questions generated by Ragas GPT-3.5 - RAG by GPT-4
{'faithfulness': 0.2087, 'answer_relevancy': 0.6278, 'context_precision': 0.3977, 'context_recall': 0.1111, 'harmfulness': 0.0000}


---

In [1]:
result

NameError: name 'result' is not defined